In [51]:
from dotenv import load_dotenv
import gradio as gr 
import os
import google.generativeai as genai
 

In [52]:
api_key = os.getenv("GEMINI_API_KEY")

In [62]:
genai.configure(api_key=api_key)

In [86]:
system_message="""You are an intelligent AI assistant  that promptly follows the instructions given by the user
"""

## Function for detecting missing keywords

In [87]:
import re

def find_missing_keywords(keywords_file_path: str, homework_file_path: str) -> list[str]:
    """
    Checks which keywords or phrases are missing from the homework file.
    Supports multi-word keyword phrases.
    
    Args:
        keywords_file_path (str): Path to the file containing keywords (one per line or comma-separated).
        homework_file_path (str): Path to the student's homework file.
    
    Returns:
        list[str]: A list of keywords/phrases that are not found in the homework.
    """
    
    with open(keywords_file_path, 'r', encoding='utf-8') as f:
        raw_keywords = f.read()
    keywords = [kw.strip().lower() for kw in re.split(r'[\n,]+', raw_keywords) if kw.strip()]
    
    with open(homework_file_path, 'r', encoding='utf-8') as f:
        homework_text = f.read().lower()
    # Removing punctuation for more robust phrase matching
    homework_text_clean = re.sub(r'[^\w\s]', '', homework_text)
    
    # Checking for presence of each keyword/phrase
    missing_keywords = []
    for keyword in keywords:
        kw_clean = re.sub(r'[^\w\s]', '', keyword.lower())
        if kw_clean not in homework_text_clean:
            missing_keywords.append(keyword)
    
    return missing_keywords


In [88]:
ans=find_missing_keywords("C:/Users/hp/Desktop/AI-Internship/Task-1/keywords.txt","C:/Users/hp/Desktop/AI-Internship/Task-1/homework2.txt")
print(ans)

['artificial intelligence', 'machine learning', 'healthcare', 'ethics', 'solar panels', 'quantum computing', 'cryptography', 'data analysis', 'ai-powered applications', 'energy efficiency', 'autonomous vehicles', 'nlp', 'ethical guidelines', 'medical image analysis', 'quantum bits']


In [89]:
from google.generativeai.types import FunctionDeclaration, Tool

In [90]:
def create_keyword_checking_tool():
    return Tool(
        function_declarations=[
            FunctionDeclaration(
                name=find_missing_keywords.__name__,
                description="Checks which keywords or phrases from a given file are missing in the student's homework file.",
                parameters={
                    "type": "object",
                    "properties": {
                        "keywords_file_path": {
                            "type": "string",
                            "description": "Path to the file containing the list of required keywords or phrases (one per line or comma-separated)."
                        },
                        "homework_file_path": {
                            "type": "string",
                            "description": "Path to the student's homework file to be checked."
                        }
                    },
                    "required": ["keywords_file_path", "homework_file_path"]
                }
            )
        ]
    )

In [91]:
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    tools=[create_keyword_checking_tool()],
    system_instruction=system_message
)


In [94]:
import gradio as gr
import google.generativeai as genai
import json

 
# -- Gradio Processing Function --
def generate_feedback(keywords_file, homework_file):
    if not keywords_file or not homework_file:
        yield "Please upload both the keywords file and the homework file."
        return

    prompt = (
        f"Use the tool to find missing keywords from the student's homework.\n"
        f"keywords_file_path: {keywords_file.name}\n"
        f"homework_file_path: {homework_file.name}\n"
        
    )

    response = model.generate_content(
        contents = prompt,
    )
        # Step 2: Extract function call from response
    parts = response.candidates[0].content.parts
    function_call = None
    for part in parts:
        if hasattr(part, "function_call"):
            function_call = part.function_call
            break

    if not function_call:
        yield "❌ No function call was found in the response."
        return

     
    if isinstance(function_call.args, str):
        args =json.loads(function_call.ars)
    else:
        args = function_call.args

    homework_file_path = args.get("homework_file_path")
    keywords_file_path = args.get("keywords_file_path")

    # Step 4: Manually call the tool
    missing_keywords = find_missing_keywords(keywords_file_path, homework_file_path)

   
    summary_prompt = (
        f"The following keywords are missing from the student's homework:\n"
        f"{', '.join(missing_keywords)}\n\n"
         "Provide a clear summary of the missing keywords and put them in groups they belong to.Add a new line for each group.Don't provide additional information or advice"
    )

    summary_response = model.generate_content(
        contents=[{"role": "user", "parts": [{"text": summary_prompt}]}],
    )

    # Step 7: Stream Gemini's explanation
    # output = summary_response.content[0].content.parts[0].text
    output = summary_response.candidates[0].content.parts[0].text

   
    yield output

In [97]:
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 AI-Powered Keyword Checker")

    with gr.Row():
        keywords_file = gr.File(label="Upload Keywords File (.txt)", file_types=[".txt"])
        homework_file = gr.File(label="Upload Homework File (.txt)", file_types=[".txt"])

    submit_btn = gr.Button("Check Missing Keywords")
    output = gr.Textbox(label="AI feedback",interactive=True)

    submit_btn.click(
        fn=generate_feedback,
        inputs=[keywords_file, homework_file],
        outputs=output,
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7880

Could not create share link. Missing file: C:\Users\hp\.cache\huggingface\gradio\frpc\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: C:\Users\hp\.cache\huggingface\gradio\frpc
